In [830]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from collections import Counter
import xgboost as xgb
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder

In [832]:
df_account_holder = pd.read_csv('account holder data.csv', index_col = 0)
df_account = pd.read_csv('account data.csv', index_col = 0)
df_mule_flag = pd.read_csv('mule flag.csv', index_col = 0)

In [834]:
df_account_holder['DateOfBirth'] = pd.to_datetime(df_account_holder['DateOfBirth'], format='%d/%m/%Y', errors='coerce')

In [836]:
print(df_account_holder.index.duplicated().sum())
print(df_account.index.duplicated().sum())
print(df_mule_flag.index.duplicated().sum())

45
0
0


In [838]:
df_account_holder = df_account_holder.drop_duplicates()

In [840]:
# Calculate age in years
today = pd.Timestamp('today')
df_account_holder['Age'] = (today - df_account_holder['DateOfBirth']).dt.days // 365.25

In [842]:
df_account_holder = df_account_holder.drop(columns = ['DateOfBirth'])

In [844]:
df_combined = pd.concat([df_account_holder, df_mule_flag], axis = 1)

In [846]:
df = df_combined[df_combined['MuleAccount'].notna()]

In [848]:
df = df.drop(columns = ['MuleAccount'])

In [850]:
df

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age
Identifier,,,,,,,,,,,,
ID_0001,Female,96534.0,NaN,44116.0,Unemployed,Single,Owner,3.0,21.0,9.0,Yes,61.0
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,NaN,Owner,2.0,9.0,10.0,No,20.0
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.0,11.0,5.0,No,49.0
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.0,19.0,4.0,No,35.0
ID_0007,NaN,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.0,11.0,6.0,Yes,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,NaN,55262.0,NaN,1299.0,Unemployed,Widowed,Renter,4.0,9.0,10.0,Yes,45.0
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.0,12.0,5.0,Yes,57.0
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,NaN,0.0,10.0,15.0,No,63.0


In [852]:
# Create encoder that will treat NaN as missing
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)

# Encode categorical columns
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = encoder.fit_transform(df[cat_cols])

In [854]:
# Apply KNN imputation
imputer = KNNImputer(n_neighbors=15)
imputed_data = imputer.fit_transform(df)

# Convert back to DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=df.columns)

# Decode back to original categories
for col, le in label_encoders.items():
    imputed_df[col] = le.inverse_transform(imputed_df[col].astype(int))

In [856]:
df_1 = imputed_df.copy()
df_1.index = df.index

In [858]:
df_1

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age
Identifier,,,,,,,,,,,,
ID_0001,Female,96534.0,545.2,44116.0,Unemployed,Single,Owner,3.000000,21.0,9.0,Yes,61.0
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,Married,Owner,2.000000,9.0,10.0,No,20.0
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.000000,11.0,5.0,No,49.0
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.000000,19.0,4.0,No,35.0
ID_0007,Female,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.000000,11.0,6.0,Yes,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,Female,55262.0,686.6,1299.0,Unemployed,Widowed,Renter,4.000000,9.0,10.0,Yes,45.0
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.000000,12.0,5.0,Yes,57.0
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,Owner,0.000000,10.0,15.0,No,63.0


In [860]:
print(df_1.min())
print(df_1.max())

Gender                         Female
Income                        20060.0
CreditScore                     300.0
LoanAmount                       60.0
EmploymentStatus             Employed
MaritalStatus                Divorced
OccupancyStatus                 Owner
NumDependents                     0.0
SocialMediaUsageHours             0.0
ShoppingFrequencyPerMonth         0.0
HealthInsuranceStatus              No
Age                              18.0
dtype: object
Gender                             Male
Income                          99973.0
CreditScore                       849.0
LoanAmount                      49839.0
EmploymentStatus             Unemployed
MaritalStatus                   Widowed
OccupancyStatus                  Renter
NumDependents                       4.0
SocialMediaUsageHours              23.0
ShoppingFrequencyPerMonth          29.0
HealthInsuranceStatus               Yes
Age                                70.0
dtype: object


In [862]:
bins = [300, 670, 740, 800, 850]
labels = ['Bad', 'Good', 'Very Good', 'Excellent']
df_1['CreditScore_Rating'] = pd.cut(df_1['CreditScore'], bins=bins, labels=labels)

In [864]:
df_1

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age,CreditScore_Rating
Identifier,,,,,,,,,,,,,
ID_0001,Female,96534.0,545.2,44116.0,Unemployed,Single,Owner,3.000000,21.0,9.0,Yes,61.0,Bad
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,Married,Owner,2.000000,9.0,10.0,No,20.0,Bad
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.000000,11.0,5.0,No,49.0,Very Good
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.000000,19.0,4.0,No,35.0,Bad
ID_0007,Female,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.000000,11.0,6.0,Yes,64.0,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,Female,55262.0,686.6,1299.0,Unemployed,Widowed,Renter,4.000000,9.0,10.0,Yes,45.0,Good
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.000000,12.0,5.0,Yes,57.0,Good
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,Owner,0.000000,10.0,15.0,No,63.0,Bad


## Credit Score:
- 300 - 669 -> bad
- 670 - 739 -> good
- 740 - 799 -> very good
- above 800 -> excellent

source: https://www.experian.com/blogs/ask-experian/credit-education/score-basics/what-is-a-good-credit-score/

In [867]:
bins = [18, 25, 35, 50, 65, 70]
labels = ['Young Adults', 'Early Career', 'Established Adults', 'Pre-Retirement', 'Retirees']
df_1['Age group'] = pd.cut(df_1['Age'], bins=bins, labels=labels)

## Standard age-based segmentation:
- 18 - 25 -> Young Adults
- 26 - 35 -> Early Career
- 36 - 50 -> Established Adults
- 51 - 65 -> Pre Retirement
- 65+     -> Retirees

In [870]:
df_1

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age,CreditScore_Rating,Age group
Identifier,,,,,,,,,,,,,,
ID_0001,Female,96534.0,545.2,44116.0,Unemployed,Single,Owner,3.000000,21.0,9.0,Yes,61.0,Bad,Pre-Retirement
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,Married,Owner,2.000000,9.0,10.0,No,20.0,Bad,Young Adults
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.000000,11.0,5.0,No,49.0,Very Good,Established Adults
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.000000,19.0,4.0,No,35.0,Bad,Early Career
ID_0007,Female,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.000000,11.0,6.0,Yes,64.0,Bad,Pre-Retirement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,Female,55262.0,686.6,1299.0,Unemployed,Widowed,Renter,4.000000,9.0,10.0,Yes,45.0,Good,Established Adults
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.000000,12.0,5.0,Yes,57.0,Good,Pre-Retirement
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,Owner,0.000000,10.0,15.0,No,63.0,Bad,Pre-Retirement


In [872]:
bins = [0000, 25, 35, 50, 65, 70]
labels = ['Young Adults', 'Early Career', 'Established Adults', 'Pre-Retirement', 'Retirees']
df_1['Age group'] = pd.cut(df_1['Age'], bins=bins, labels=labels)

In [874]:
bins = [20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]
df_1['Income_bin'] = pd.cut(df_1['Income'], bins=bins)

In [876]:
df_1

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age,CreditScore_Rating,Age group,Income_bin
Identifier,,,,,,,,,,,,,,,
ID_0001,Female,96534.0,545.2,44116.0,Unemployed,Single,Owner,3.000000,21.0,9.0,Yes,61.0,Bad,Pre-Retirement,"(90000, 100000]"
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,Married,Owner,2.000000,9.0,10.0,No,20.0,Bad,Young Adults,"(70000, 80000]"
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.000000,11.0,5.0,No,49.0,Very Good,Established Adults,"(80000, 90000]"
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.000000,19.0,4.0,No,35.0,Bad,Early Career,"(90000, 100000]"
ID_0007,Female,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.000000,11.0,6.0,Yes,64.0,Bad,Pre-Retirement,"(70000, 80000]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,Female,55262.0,686.6,1299.0,Unemployed,Widowed,Renter,4.000000,9.0,10.0,Yes,45.0,Good,Established Adults,"(50000, 60000]"
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.000000,12.0,5.0,Yes,57.0,Good,Pre-Retirement,"(90000, 100000]"
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,Owner,0.000000,10.0,15.0,No,63.0,Bad,Pre-Retirement,"(30000, 40000]"


In [878]:
bins = [0, 10000, 20000, 30000, 40000, 50000, 60000]
df_1['LoanAmount_bin'] = pd.cut(df_1['LoanAmount'], bins=bins)

In [880]:
bins = [0, 5, 10, 15, 20, 25]
df_1['SocialMediaUsageHours_bin'] = pd.cut(df_1['SocialMediaUsageHours'], bins=bins)

In [882]:
bins = [0, 5, 10, 15, 20, 25, 30]
df_1['ShoppingFrequencyPerMonth_bin'] = pd.cut(df_1['ShoppingFrequencyPerMonth'], bins=bins)

In [884]:
df_1

,Gender,Income,CreditScore,LoanAmount,EmploymentStatus,MaritalStatus,OccupancyStatus,NumDependents,SocialMediaUsageHours,ShoppingFrequencyPerMonth,HealthInsuranceStatus,Age,CreditScore_Rating,Age group,Income_bin,LoanAmount_bin,SocialMediaUsageHours_bin,ShoppingFrequencyPerMonth_bin
Identifier,,,,,,,,,,,,,,,,,,
ID_0001,Female,96534.0,545.2,44116.0,Unemployed,Single,Owner,3.000000,21.0,9.0,Yes,61.0,Bad,Pre-Retirement,"(90000, 100000]","(40000, 50000]","(20, 25]","(5, 10]"
ID_0002,Male,75106.0,390.0,9782.0,Unemployed,Married,Owner,2.000000,9.0,10.0,No,20.0,Bad,Young Adults,"(70000, 80000]","(0, 10000]","(5, 10]","(5, 10]"
ID_0005,Male,83230.0,750.0,28557.0,Employed,Single,Owner,1.000000,11.0,5.0,No,49.0,Very Good,Established Adults,"(80000, 90000]","(20000, 30000]","(10, 15]","(0, 5]"
ID_0006,Male,94055.0,664.0,5798.0,Employed,Married,Renter,4.000000,19.0,4.0,No,35.0,Bad,Early Career,"(90000, 100000]","(0, 10000]","(15, 20]","(0, 5]"
ID_0007,Female,76373.0,650.0,9627.0,Unemployed,Divorced,Renter,2.000000,11.0,6.0,Yes,64.0,Bad,Pre-Retirement,"(70000, 80000]","(0, 10000]","(10, 15]","(5, 10]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_0996,Female,55262.0,686.6,1299.0,Unemployed,Widowed,Renter,4.000000,9.0,10.0,Yes,45.0,Good,Established Adults,"(50000, 60000]","(0, 10000]","(5, 10]","(5, 10]"
ID_0997,Female,98213.0,725.0,39469.0,Unemployed,Married,Renter,3.000000,12.0,5.0,Yes,57.0,Good,Pre-Retirement,"(90000, 100000]","(30000, 40000]","(10, 15]","(0, 5]"
ID_0998,Male,38268.0,324.0,4994.0,Retired,Divorced,Owner,0.000000,10.0,15.0,No,63.0,Bad,Pre-Retirement,"(30000, 40000]","(0, 10000]","(5, 10]","(10, 15]"
